# ETL 

Lo primero que hacemos es llamar las funciones: `Fitizens_libraries` es la carpeta en la que se encuentan los archivos .py con las funciones:

In [ ]:
from fitizens_libraries.load_and_process_training_data import load_training_data
from fitizens_libraries.load_timeseries import load_timeseries_data
from custom_libraries.merge_data import merge_data
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from fitizens_libraries.plot_labeled_sequences import plot_labeled_sequence
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from matplotlib import pyplot
from fitizens_libraries.build_dataframe_from_list_of_signals import build_dataframe
import pandas as pd
import os
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, auc, roc_auc_score
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc

Para el proceso de ETL lo que haremos será utilizar la función de `load_training_data`:
1. Basicamente nos busca los archivos JSON en la carpeta zip en la que se encuentren y los carga.
2. Intenta buscar repeticiones falsas y verdaderas del ejercicio 
3. Se filtran las repeticiones falsas para que tengan una duración dentro del rango de las repeticiones reales.

Es importante tener en cuenta que la función me exige unos hiperparametros obligatorios que tengo que indicar:

- signals : list of str
    List of signals to include in the dataframe. The signals must be present in the input data
- target_exercise : str
        Exercise to detect repetitions
- other_exercises : list of str
        List of exercises to use as negative examples

In [ ]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True) #Referenciamos la carpeta LABELED en la que están las carpetas zip con los json
#Ahora voy a iterar en esa carpeta LABELED para obtener la ruta de los archivos, que es el LABELED/NOMBRE y eso lo guardo en una lista
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
#Indico mi target
target_exercise="SQUAT"

In [ ]:
len(file_names)

In [ ]:
data,wk = load_training_data(filelist=file_names,
                         signals= signals,
                          target_exercise=target_exercise, other_exercises=[], is_peak_minima=True)

In [ ]:
#df2=build_dataframe(data)

In [ ]:
#df2.head()

In [ ]:
data[0]

In [ ]:
#Revisamos
#data
len(data)

In [ ]:
data[1]['target']

In [ ]:
df = merge_data(data)
len(df)

In [ ]:
df.head()

In [ ]:
df.columns

Ahora voy a convertir esto en un problema de clasificacion binario; para ello, voy a crear una columna de exercise, en la que segun la columna `exercising_periods` me diga si hay o no un squad. Si esa columna tiene un cero, indica que no hay squad, de lo contrario es un squad.

In [ ]:
def nueva_columna(exercise):
    if exercise == 0:
        return 'no exercise'
    else:
        return 'squad'
df['squad'] = df['exercising_periods'].apply(nueva_columna)

In [ ]:
df.head()

In [ ]:
df.info()

# EDA

In [ ]:
#revisar porcentaje de valores nulos por columnas
((df.isnull().sum())/len(df))*100

In [ ]:
sns.countplot(x=df['squad'], label = "squad")

In [ ]:
df.index

In [ ]:
fig = px.line(df, x=df.index, y='linAccZ', title='Time serie of exercise linAccZ',color='squad')
fig.show()

In [ ]:
fig = px.box(df, y="linAccZ", color="squad",title = 'Distribution of linAccZ vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="linAccY", color="squad",title = 'Distribution of linAccY vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="linAccX", color="squad",title = 'Distribution of linAccX vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="accZ_mod", color="squad",title = 'Distribution of accZ_mod vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="accX_mod", color="squad",title = 'Distribution of accX_mod vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
fig = px.box(df, y="accY_mod", color="squad",title = 'Distribution of accY_mod vs target variable')
fig.update_traces(quartilemethod="exclusive") 
fig.show()

In [ ]:
timeseries, labels_ranges = load_timeseries_data(filelist=file_names, signals=signals, is_peak_minima=True)

In [ ]:
timeseries.columns

In [ ]:
plot_labeled_sequence(timeseries, labels_ranges[0:5])

In [ ]:
labels_ranges

In [ ]:
y = df['exercising_periods']

num_lags=24 # to discuss

plt.plot(y)
plt.show()
plot_acf(y,lags=num_lags)
plt.show()
plot_pacf(y,lags=num_lags,method="ols")
plt.show()

In [ ]:
import statsmodels as sm

# Ho: the process is not stationary. We need, at least, a unit root
# H1: the process is stationary. We have to check different models (lags)


adf_test = sm.tsa.stattools.adfuller(y,maxlag=10) 
# print("adf_test", adf_test) # first output "The test statistic", second output "p-value"
print("ADF test for the original series")
print("Statistic Value:" , adf_test[0])
print("p-value:" , adf_test[1])
# print(sm.__version__)

In [ ]:
prim_1000 = df.head(1000)

In [ ]:
fig = px.line(prim_1000, x=prim_1000.index, y='linAccZ', title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
siguientes_2000 = df.tail(len(df) - 1000).head(1500)

In [ ]:
fig = px.line(siguientes_2000, x=siguientes_2000.index, y='linAccZ', title='Time serie of exercise linAccZ')
fig.show()

# Data prep for model

In [ ]:
df = merge_data(data)
len(df)

In [ ]:
serie = pd.concat([item['series'] for item in data])

In [ ]:
serie.head()

In [ ]:
len(serie)

In [ ]:
frames = []
target = []
for item in data:
    frames.append(item['series'])
    target.append(item['target'])

df = pd.concat(frames)
#df['target'] = target

In [ ]:
len(frames)

In [ ]:
len(target)

In [ ]:
Counter(target)

In [ ]:
#vamos a crear un dataframe con los promedios
promedios_df = pd.DataFrame()

# Iterar sobre cada diccionario en la lista
for diccionario in data:
    # Convertir el diccionario a un DataFrame
    df = diccionario['series']

    # Calcular el promedio de cada columna y agregarlo al DataFrame de promedios
    promedio_serie = df.mean()
    promedios_df = promedios_df.append(promedio_serie,ignore_index=True)

# Agregar la columna 'target' al DataFrame de promedios
promedios_df['target'] = [diccionario['target'] for diccionario in data]

In [ ]:
promedios_df.head()

In [ ]:
#verificamos la longitud del dataframe que coincida con los 3805
len(promedios_df)

In [ ]:
#verificamos que el target siga desbalanceado y coincidan los numeros
promedios_df.groupby('target').size()

In [ ]:
promedios_df.info()

In [ ]:
promedios_df.head()

In [ ]:
sns.countplot(x=promedios_df['target'], label = "squad")

In [ ]:
fig = px.box(promedios_df, y="linAccZ", color="target",title = 'Distribution of linAccZ vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="linAccZ_orig", color="target",title = 'Distribution of linAccZ_orig vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="linAccY", color="target",title = 'Distribution of linAccY vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="linAccY_orig", color="target",title = 'Distribution of linAccY_orig vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="linAccX", color="target",title = 'Distribution of linAccX vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="linAccX_orig", color="target",title = 'Distribution of linAccX_orig vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="accZ_mod", color="target",title = 'Distribution of accZ_mod vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="accZ_orig", color="target",title = 'Distribution of accZ_orig vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="accX_mod", color="target",title = 'Distribution of accX_mod vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(promedios_df, y="accY_mod", color="target",title = 'Distribution of accY_mod vs target variable')
fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
sns.pairplot(promedios_df, hue= 'target',vars=["linAccX", "linAccY", "linAccZ"])

In [ ]:
plt.figure(figsize=(30, 30))
sns.heatmap( promedios_df.corr(), annot = True, cmap ="coolwarm", linewidths = .5)

In [ ]:
X = promedios_df.drop('target',axis=1)
y = promedios_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99,stratify=y)
display(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

## SMOTE for balance my data

In [ ]:
smote = SMOTE( random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
#check the new sample
import numpy as np
np.unique(y_train_resampled,return_counts=True)

## Feature selection

In [ ]:
selected = SelectKBest(score_func=f_classif, k=9)
X_train_selected=selected.fit_transform(X_train_resampled, y_train_resampled)
X_test_selected = selected.transform(X_test) #I have to choose also for the test

In [ ]:
selected.get_feature_names_out()

In [ ]:
X_train_selected

In [ ]:
scores = pd.Series(selected.scores_, index=X.columns)
scores = scores.sort_values(ascending=False)
px.bar(scores, template="none", title="F-Score of features with casual as dependent variable")

In [ ]:
# Scale Data
scaler = StandardScaler()
X_train_full = scaler.fit_transform(X_train_selected) #I only use the selected variables
# apply stanrdadization also to the test
X_test_full = scaler.transform(X_test_selected)

In [ ]:
# ahora tengo que hacer la codificacion de la variable target 
label_encoder = LabelEncoder()
# Aplica la transformación a la variable y
y_train_full = label_encoder.fit_transform(y_train_resampled)
y_test= label_encoder.transform(y_test)

# Models

In [ ]:
results_hard = {}
results_soft = {}

## SVM

In [ ]:
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
svm = SVC(probability=True)

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}


# Realizar la búsqueda aleatoria de hiperparámetros
random_search_svm = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_grid,
    n_iter=5,  
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
random_search_svm.fit(X_train_full, y_train_full)

In [ ]:
print(random_search_svm.best_params_)

In [ ]:
#First evaluate on train
proba_train = random_search_svm.predict_proba(X_train_full)
pred_train = random_search_svm.predict(X_train_full)
print(classification_report(y_train_full,pred_train))

In [ ]:
#now I will check with the test
proba_test = random_search_svm.predict_proba(X_test_full)
pred_test = random_search_svm.predict(X_test_full)
print(classification_report(y_test,pred_test))

## XGBOOST

In [ ]:
import xgboost as xgb
xgb = xgb.XGBClassifier()

In [ ]:
param_dist = {
    'n_estimators': range(10, 90, 5),
    'max_depth': range(3, 40, 2),
    'min_child_weight': range(1, 10),  
    'gamma': [0, 0.1, 0.2, 0.3],  
}

# Realizar la búsqueda aleatoria de hiperparámetros
random_search_XGB = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10, 
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)

In [ ]:
#para este modelo uso datos sin escalar 
random_search_XGB.fit(X_train_selected, y_train_full)

In [ ]:
print(random_search_XGB.best_params_)

In [ ]:
#First evaluate on train
proba_train_XGB = random_search_XGB.predict_proba(X_train_selected)
pred_train_XGB = random_search_XGB.predict(X_train_selected)
print(classification_report(y_train_full,pred_train_XGB))

In [ ]:
#now I will check with the test
proba_test_XGB = random_search_XGB.predict_proba(X_test_selected)
pred_test_XGB = random_search_XGB.predict(X_test_selected)
print(classification_report(y_test,pred_test_XGB))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
param_dist_NB = {
    'priors': [[0.2, 0.8], [0.5, 0.5], [0.8, 0.2],[0.3, 0.7],[0.7, 0.3]]
}


random_search_NB = RandomizedSearchCV(
    estimator=gaus,
    param_distributions=param_dist_NB,
    n_iter=5,  
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)

In [ ]:
# Entrenar el modelo con la búsqueda aleatoria de hiperparámetros
random_search_NB.fit(X_train_full, y_train_full)

In [ ]:
print(random_search_NB.best_params_)

In [ ]:
#First evaluate on train
proba_train_NB = random_search_NB.predict_proba(X_train_full)
pred_train_NB = random_search_NB.predict(X_train_full)
print(classification_report(y_train_full,pred_train_NB))

In [ ]:
#now I will check with the test
proba_test_NB = random_search_NB.predict_proba(X_test_full)
pred_test_NB = random_search_NB.predict(X_test_full)
print(classification_report(y_test,pred_test_NB))

## Model evaluation

In [ ]:
pred_SV = random_search_svm.predict(X_test_full)
proba_SV = random_search_svm.predict_proba(X_test_full)
results_hard["Support_Vector"] = pred_SV
results_soft["Support_Vector"] = proba_SV[:,1]

proba_XGB = random_search_XGB.predict_proba(X_test_selected)
pred_XGB = random_search_XGB.predict(X_test_selected)
results_hard["XGBOOST"] = pred_XGB
results_soft["XGBOOST"] = proba_XGB[:,1]

proba_NB = random_search_NB.predict_proba(X_test_full)
pred_NB = random_search_NB.predict(X_test_full)
results_hard["Naive_Bayes"] = pred_NB
results_soft["Naive_Bayes"] = proba_NB[:,1]

results_hard = pd.DataFrame(results_hard)
results_soft = pd.DataFrame(results_soft)

In [ ]:
metrics = {}

metrics["Accuracy"] = {
    "Naive_Bayes": accuracy_score(y_test, results_hard.Naive_Bayes),
    "Support_Vector": accuracy_score(y_test, results_hard.Support_Vector),
    "XGBOOST": accuracy_score(y_test, results_hard.XGBOOST)
}
metrics["Precision"] = {
    "Naive_Bayes": precision_score(y_test, results_hard.Naive_Bayes),
    "Support_Vector": precision_score(y_test, results_hard.Support_Vector),
    "XGBOOST": precision_score(y_test, results_hard.XGBOOST)
}
metrics["Recall"] = {
    "Naive_Bayes": recall_score(y_test, results_hard.Naive_Bayes),
    "Support_Vector": recall_score(y_test, results_hard.Support_Vector),
    "XGBOOST": recall_score(y_test, results_hard.XGBOOST)
}
metrics["F1"] = {
    "Naive_Bayes": f1_score(y_test, results_hard.Naive_Bayes),
    "Support_Vector": f1_score(y_test, results_hard.Support_Vector),
    "XGBOOST": f1_score(y_test, results_hard.XGBOOST)
}

metrics = pd.DataFrame(metrics)
metrics

## ROC Curves

In [ ]:
# Datos de FPR y TPR para los tres modelos 
fpr_Naive_Bayes,tpr_Naive_Bayes,_ = roc_curve(y_test, results_soft.Naive_Bayes)
fpr_Support_Vector,tpr_Support_Vector,_ = roc_curve(y_test, results_soft.Support_Vector)
fpr_XGBOOST,tpr_XGBOOST,_ = roc_curve(y_test, results_soft.XGBOOST)

# Calcular el área bajo la curva ROC (AUC) para cada modelo
auc_Naive_Bayes = auc(fpr_Naive_Bayes,tpr_Naive_Bayes)
auc_Support_Vector = auc(fpr_Support_Vector,tpr_Support_Vector)
auc_XGBOOST = auc(fpr_XGBOOST,tpr_XGBOOST)

In [ ]:
# Crear la gráfica ROC
plt.figure(figsize=(8, 6))

# Graficar las curvas ROC para los tres modelos
plt.plot(fpr_Naive_Bayes,tpr_Naive_Bayes, label=f'Naive bayes (AUC = {auc_Naive_Bayes:.2f})')
plt.plot(fpr_XGBOOST,tpr_XGBOOST, label=f'Xgboost (AUC = {auc_XGBOOST:.2f})')
plt.plot(fpr_Support_Vector,tpr_Support_Vector, label=f'Support Vector (AUC = {auc_Support_Vector:.2f})')


# Configurar la gráfica
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Línea diagonal para referencia
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC de Modelos')
plt.legend(loc="lower right")

# Mostrar la gráfica
plt.show()

# Try out models with other features

Para este caso solo voy a usar las variables entregadas por el sensor: 
- Raw acceleration.
- Angular Velocity.
- Magnetic Field.
- Linear Acceleration.

In [ ]:
X = promedios_df.drop('target',axis=1)
y = promedios_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99,stratify=y)
display(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
smote = SMOTE( random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
selected_columns = ['accX_orig', 'accY_orig', 'accZ_orig', 'gyroX_orig', 'gyroY_orig',
                    'gyroZ_orig', 'magnX_orig', 'magnY_orig', 'magnZ_orig', 
                    'linAccX_orig', 'linAccY_orig', 'linAccZ_orig', 'accX', 'accY',
                    'accZ', 'gyroX', 'gyroY', 'gyroZ', 'magnX', 'magnY', 'magnZ', 
                    'linAccX', 'linAccY', 'linAccZ', 'accX_mod', 'accY_mod', 'accZ_mod', 
                    'gyroX_mod', 'gyroY_mod', 'gyroZ_mod', 'magnX_mod', 'magnY_mod', 
                    'magnZ_mod', 'linAccX_mod', 'linAccY_mod', 'linAccZ_mod']

X_train2_ns = X_train_resampled[selected_columns].copy()
X_test2_ns = X_test[selected_columns].copy()

In [ ]:
selected2 = SelectKBest(score_func=f_classif, k=20)
X_train2_selected=selected2.fit_transform(X_train2_ns, y_train_resampled)
X_test2_selected = selected2.transform(X_test2_ns) #I have to choose also for the test
selected2.get_feature_names_out()

In [ ]:
# Scale Data
scaler2 = StandardScaler()
X_train2_full = scaler2.fit_transform(X_train2_selected) #I only use the selected variables
# apply stanrdadization also to the test
X_test2_full = scaler2.transform(X_test2_selected)

In [ ]:
# ahora tengo que hacer la codificacion de la variable target 
label_encoder = LabelEncoder()
# Aplica la transformación a la variable y
y_train_full = label_encoder.fit_transform(y_train_resampled)
y_test= label_encoder.transform(y_test)

In [ ]:
results_hard2 = {}
results_soft2 = {}

## SVM

In [ ]:
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
svm = SVC(probability=True)
param_grid = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}


# Realizar la búsqueda aleatoria de hiperparámetros
random_search_svm = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_grid,
    n_iter=5,  
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
random_search_svm.fit(X_train2_selected, y_train_full)

In [ ]:
print(random_search_svm.best_params_)

In [ ]:
#First evaluate on train
proba_train2 = random_search_svm.predict_proba(X_train2_selected)
pred_train2 = random_search_svm.predict(X_train2_selected)
print(classification_report(y_train_full,pred_train2))

In [ ]:
#now I will check with the test
proba_test2 = random_search_svm.predict_proba(X_test2_full)
pred_test2 = random_search_svm.predict(X_test2_full)
print(classification_report(y_test,pred_test2))

## XGBOOST

In [ ]:
import xgboost as xgb
xgb = xgb.XGBClassifier()
param_dist = {
    'n_estimators': range(10, 90, 5),
    'max_depth': range(3, 40, 2),
    'min_child_weight': range(1, 10),  
    'gamma': [0, 0.1, 0.2, 0.3],  
}

# Realizar la búsqueda aleatoria de hiperparámetros
random_search_XGB = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10, 
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
#para este modelo uso datos sin escalar 
random_search_XGB.fit(X_train2_selected, y_train_full)
print(random_search_XGB.best_params_)

In [ ]:
#First evaluate on train
proba_train_XGB2 = random_search_XGB.predict_proba(X_train2_selected)
pred_train_XGB2 = random_search_XGB.predict(X_train2_selected)
print(classification_report(y_train_full,pred_train_XGB2))

In [ ]:
#now I will check with the test
proba_test_XGB2 = random_search_XGB.predict_proba(X_test2_selected)
pred_test_XGB2 = random_search_XGB.predict(X_test2_selected)
print(classification_report(y_test,pred_test_XGB2))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
param_dist_NB = {
    'priors': [[0.2, 0.8], [0.5, 0.5], [0.8, 0.2],[0.3, 0.7],[0.7, 0.3]]
}


random_search_NB = RandomizedSearchCV(
    estimator=gaus,
    param_distributions=param_dist_NB,
    n_iter=5,  
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
# Entrenar el modelo con la búsqueda aleatoria de hiperparámetros
random_search_NB.fit(X_train2_selected, y_train_full)

In [ ]:
print(random_search_NB.best_params_)

In [ ]:
#First evaluate on train
proba_train_NB2 = random_search_NB.predict_proba(X_train2_selected)
pred_train_NB2 = random_search_NB.predict(X_train2_selected)
print(classification_report(y_train_full,pred_train_NB2))

In [ ]:
#now I will check with the test
proba_test_NB2 = random_search_NB.predict_proba(X_test2_full)
pred_test_NB2 = random_search_NB.predict(X_test2_full)
print(classification_report(y_test,pred_test_NB2))

## Model evaluation

In [ ]:
pred_SV2 = random_search_svm.predict(X_test2_full)
proba_SV2 = random_search_svm.predict_proba(X_test2_full)
results_hard2["Support_Vector"] = pred_SV2
results_soft2["Support_Vector"] = proba_SV2[:,1]

proba_XGB2 = random_search_XGB.predict_proba(X_test2_selected)
pred_XGB2 = random_search_XGB.predict(X_test2_selected)
results_hard2["XGBOOST"] = pred_XGB2
results_soft2["XGBOOST"] = proba_XGB2[:,1]

proba_NB2 = random_search_NB.predict_proba(X_test2_full)
pred_NB2 = random_search_NB.predict(X_test2_full)
results_hard2["Naive_Bayes"] = pred_NB2
results_soft2["Naive_Bayes"] = proba_NB2[:,1]

results_hard2 = pd.DataFrame(results_hard2)
results_soft2 = pd.DataFrame(results_soft2)

In [ ]:
metrics2 = {}

metrics2["Accuracy"] = {
    "Naive_Bayes": accuracy_score(y_test, results_hard2.Naive_Bayes),
    "Support_Vector": accuracy_score(y_test, results_hard2.Support_Vector),
    "XGBOOST": accuracy_score(y_test, results_hard2.XGBOOST)
}
metrics2["Precision"] = {
    "Naive_Bayes": precision_score(y_test, results_hard2.Naive_Bayes),
    "Support_Vector": precision_score(y_test, results_hard2.Support_Vector),
    "XGBOOST": precision_score(y_test, results_hard2.XGBOOST)
}
metrics2["Recall"] = {
    "Naive_Bayes": recall_score(y_test, results_hard2.Naive_Bayes),
    "Support_Vector": recall_score(y_test, results_hard2.Support_Vector),
    "XGBOOST": recall_score(y_test, results_hard2.XGBOOST)
}
metrics2["F1"] = {
    "Naive_Bayes": f1_score(y_test, results_hard2.Naive_Bayes),
    "Support_Vector": f1_score(y_test, results_hard2.Support_Vector),
    "XGBOOST": f1_score(y_test, results_hard2.XGBOOST)
}

metrics2 = pd.DataFrame(metrics2)
metrics2

## ROC Curves

In [ ]:
# Datos de FPR y TPR para los tres modelos 
fpr_Naive_Bayes2,tpr_Naive_Bayes2,_ = roc_curve(y_test, results_soft2.Naive_Bayes)
fpr_Support_Vector2,tpr_Support_Vector2,_ = roc_curve(y_test, results_soft2.Support_Vector)
fpr_XGBOOST2,tpr_XGBOOST2,_ = roc_curve(y_test, results_soft2.XGBOOST)

# Calcular el área bajo la curva ROC (AUC) para cada modelo
auc_Naive_Bayes2 = auc(fpr_Naive_Bayes2,tpr_Naive_Bayes2)
auc_Support_Vector2 = auc(fpr_Support_Vector2,tpr_Support_Vector2)
auc_XGBOOST2 = auc(fpr_XGBOOST2,tpr_XGBOOST2)

In [ ]:
# Crear la gráfica ROC
plt.figure(figsize=(8, 6))

# Graficar las curvas ROC para los tres modelos
plt.plot(fpr_Naive_Bayes2,tpr_Naive_Bayes2, label=f'Naive bayes (AUC = {auc_Naive_Bayes2:.2f})')
plt.plot(fpr_XGBOOST2,tpr_XGBOOST2, label=f'Xgboost (AUC = {auc_XGBOOST2:.2f})')
plt.plot(fpr_Support_Vector2,tpr_Support_Vector2, label=f'Support Vector (AUC = {auc_Support_Vector2:.2f})')


# Configurar la gráfica
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Línea diagonal para referencia
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC de Modelos')
plt.legend(loc="lower right")

# Mostrar la gráfica
plt.show()

# Nuevas variables

Ahora voy a probar con el promedio, desv. estandar y la mediana

In [ ]:
# Crear un dataframe con los promedios, desviación estándar y mediana
nuevo_df = pd.DataFrame()

# Iterar sobre cada diccionario en la lista
for diccionario in data:
    # Convertir el diccionario a un DataFrame
    df = diccionario['series']

    # Calcular el promedio, la desviación estándar y la mediana de cada columna y agregarlos al DataFrame de promedios
    promedio_serie = df.mean()
    std_serie = df.std()
    median_serie = df.median()
    nuevo_df = nuevo_df.append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

# Agregar la columna 'target' al DataFrame de promedios
nuevo_df['target'] = [diccionario['target'] for diccionario in data]

In [ ]:
nuevo_df.head()

In [ ]:
list(nuevo_df.columns)

In [ ]:
#verificamos la longitud del dataframe que coincida con los 3805
len(nuevo_df)

In [ ]:
#verificamos que el target siga desbalanceado y coincidan los numeros
nuevo_df.groupby('target').size()

In [ ]:
sns.countplot(x=nuevo_df['target'], label = "squad")

In [ ]:
fig = px.box(nuevo_df, y=["mean_linAccZ", "std_linAccZ", "median_linAccZ"], color="target", 
             labels={"value": "Acceleration", "target": "Target"},
             title='Distribution of Acceleration vs Target Variable')

fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(nuevo_df, y=["mean_accZ_orig", "std_accZ_orig", "median_accZ_orig"], color="target", 
             labels={"value": "Acceleration", "target": "Target"},
             title='Distribution of Acceleration vs Target Variable')

fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(nuevo_df, y=["mean_accY_orig", "std_accY_orig", "median_accY_orig"], color="target", 
             labels={"value": "Acceleration", "target": "Target"},
             title='Distribution of Acceleration vs Target Variable')

fig.update_traces(quartilemethod="exclusive")
fig.show()

In [ ]:
fig = px.box(nuevo_df, y=["mean_gyroZ_orig", "std_gyroZ_orig", "median_gyroZ_orig"], color="target", 
             labels={"value": "Acceleration", "target": "Target"},
             title='Distribution of Acceleration vs Target Variable')

fig.update_traces(quartilemethod="exclusive")
fig.show()

## Prep data

In [ ]:
X = nuevo_df.drop('target',axis=1)
y = nuevo_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99,stratify=y)
display(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
smote = SMOTE( random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
selected_columns = [
    'mean_accX_orig', 'mean_accY_orig', 'mean_accZ_orig', 'mean_gyroX_orig', 'mean_gyroY_orig', 'mean_gyroZ_orig',
    'mean_magnX_orig', 'mean_magnY_orig', 'mean_magnZ_orig', 'mean_linAccX_orig', 'mean_linAccY_orig', 'mean_linAccZ_orig',
    'mean_accX', 'mean_accY', 'mean_accZ', 'mean_gyroX', 'mean_gyroY', 'mean_gyroZ',
    'mean_magnX', 'mean_magnY', 'mean_magnZ', 'mean_linAccX', 'mean_linAccY', 'mean_linAccZ', 'mean_accX_mod',
    'mean_accY_mod', 'mean_accZ_mod', 'mean_gyroX_mod', 'mean_gyroY_mod', 'mean_gyroZ_mod', 'mean_magnX_mod',
    'mean_magnY_mod', 'mean_magnZ_mod', 'mean_linAccX_mod', 'mean_linAccY_mod', 'mean_linAccZ_mod',
    'std_accX_orig', 'std_accY_orig', 'std_accZ_orig', 'std_gyroX_orig', 'std_gyroY_orig', 'std_gyroZ_orig',
    'std_magnX_orig', 'std_magnY_orig', 'std_magnZ_orig', 'std_linAccX_orig', 'std_linAccY_orig', 'std_linAccZ_orig',
    'std_accX', 'std_accY', 'std_accZ', 'std_gyroX', 'std_gyroY', 'std_gyroZ', 'std_magnX',
    'std_magnY', 'std_magnZ', 'std_linAccX', 'std_linAccY', 'std_linAccZ', 'std_accX_mod', 'std_accY_mod', 'std_accZ_mod',
    'std_gyroX_mod', 'std_gyroY_mod', 'std_gyroZ_mod', 'std_magnX_mod', 'std_magnY_mod', 'std_magnZ_mod',
    'std_linAccX_mod', 'std_linAccY_mod', 'std_linAccZ_mod', 'median_accX_orig', 'median_accY_orig',
    'median_accZ_orig', 'median_gyroX_orig', 'median_gyroY_orig', 'median_gyroZ_orig', 'median_magnX_orig',
    'median_magnY_orig', 'median_magnZ_orig', 'median_linAccX_orig', 'median_linAccY_orig', 'median_linAccZ_orig',
    'median_accX', 'median_accY', 'median_accZ', 'median_gyroX', 'median_gyroY',
    'median_gyroZ', 'median_magnX', 'median_magnY', 'median_magnZ', 'median_linAccX', 'median_linAccY',
    'median_linAccZ', 'median_accX_mod', 'median_accY_mod', 'median_accZ_mod', 'median_gyroX_mod', 'median_gyroY_mod',
    'median_gyroZ_mod', 'median_magnX_mod', 'median_magnY_mod', 'median_magnZ_mod', 'median_linAccX_mod',
    'median_linAccY_mod', 'median_linAccZ_mod'
]

X_train3_ns = X_train_resampled[selected_columns].copy()
X_test3_ns = X_test[selected_columns].copy()

In [ ]:
selected2 = SelectKBest(score_func=f_classif, k=30)
X_train3_selected=selected2.fit_transform(X_train3_ns, y_train_resampled)
X_test3_selected = selected2.transform(X_test3_ns) #I have to choose also for the test
selected2.get_feature_names_out()

In [ ]:
# Scale Data
scaler3 = StandardScaler()
X_train3_full = scaler3.fit_transform(X_train3_selected) #I only use the selected variables
# apply stanrdadization also to the test
X_test3_full = scaler3.transform(X_test3_selected)

In [ ]:
# ahora tengo que hacer la codificacion de la variable target 
label_encoder = LabelEncoder()
# Aplica la transformación a la variable y
y_train_full = label_encoder.fit_transform(y_train_resampled)
y_test= label_encoder.transform(y_test)

In [ ]:
results_hard3 = {}
results_soft3 = {}

## SVM

In [ ]:
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
svm = SVC(probability=True)
param_grid = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}


# Realizar la búsqueda aleatoria de hiperparámetros
random_search_svm = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_grid,
    n_iter=5,  
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
random_search_svm.fit(X_train3_selected, y_train_full)

In [ ]:
print(random_search_svm.best_params_)

In [ ]:
#First evaluate on train
proba_train3 = random_search_svm.predict_proba(X_train3_selected)
pred_train3 = random_search_svm.predict(X_train3_selected)
print(classification_report(y_train_full,pred_train3))

In [ ]:
#now I will check with the test
proba_test3 = random_search_svm.predict_proba(X_test3_full)
pred_test3 = random_search_svm.predict(X_test3_full)
print(classification_report(y_test,pred_test3))

## XGBOOST

In [ ]:
import xgboost as xgb
xgb = xgb.XGBClassifier()
param_dist = {
    'n_estimators': range(10, 90, 5),
    'max_depth': range(3, 40, 2),
    'min_child_weight': range(1, 10),  
    'gamma': [0, 0.1, 0.2, 0.3],  
}

# Realizar la búsqueda aleatoria de hiperparámetros
random_search_XGB = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10, 
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
#para este modelo uso datos sin escalar 
random_search_XGB.fit(X_train3_selected, y_train_full)
print(random_search_XGB.best_params_)

In [ ]:
#First evaluate on train
proba_train_XGB3 = random_search_XGB.predict_proba(X_train3_selected)
pred_train_XGB3 = random_search_XGB.predict(X_train3_selected)
print(classification_report(y_train_full,pred_train_XGB3))

In [ ]:
#now I will check with the test
proba_test_XGB3 = random_search_XGB.predict_proba(X_test3_selected)
pred_test_XGB3 = random_search_XGB.predict(X_test3_selected)
print(classification_report(y_test,pred_test_XGB3))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
param_dist_NB = {
    'priors': [[0.2, 0.8], [0.5, 0.5], [0.8, 0.2],[0.3, 0.7],[0.7, 0.3]]
}


random_search_NB = RandomizedSearchCV(
    estimator=gaus,
    param_distributions=param_dist_NB,
    n_iter=5,  
    scoring="roc_auc",  
    n_jobs=7,
    return_train_score=True
)
# Entrenar el modelo con la búsqueda aleatoria de hiperparámetros
random_search_NB.fit(X_train3_selected, y_train_full)

In [ ]:
print(random_search_NB.best_params_)

In [ ]:
#First evaluate on train
proba_train_NB3 = random_search_NB.predict_proba(X_train3_selected)
pred_train_NB3 = random_search_NB.predict(X_train3_selected)
print(classification_report(y_train_full,pred_train_NB3))

In [ ]:
#now I will check with the test
proba_test_NB3 = random_search_NB.predict_proba(X_test3_full)
pred_test_NB3 = random_search_NB.predict(X_test3_full)
print(classification_report(y_test,pred_test_NB3))

## Model evaluation

In [ ]:
#pred_SV3 = random_search_svm.predict(X_test3_full)
#proba_SV3 = random_search_svm.predict_proba(X_test3_full)
#results_hard3["Support_Vector"] = pred_SV3
#results_soft3["Support_Vector"] = proba_SV3[:,1]

proba_XGB3 = random_search_XGB.predict_proba(X_test3_selected)
pred_XGB3 = random_search_XGB.predict(X_test3_selected)
results_hard3["XGBOOST"] = pred_XGB3
results_soft3["XGBOOST"] = proba_XGB3[:,1]

proba_NB3 = random_search_NB.predict_proba(X_test3_full)
pred_NB3 = random_search_NB.predict(X_test3_full)
results_hard3["Naive_Bayes"] = pred_NB3
results_soft3["Naive_Bayes"] = proba_NB3[:,1]

results_hard3 = pd.DataFrame(results_hard3)
results_soft3 = pd.DataFrame(results_soft3)

In [ ]:
metrics3 = {}

metrics3["Accuracy"] = {
    "Naive_Bayes": accuracy_score(y_test, results_hard3.Naive_Bayes),
    #"Support_Vector": accuracy_score(y_test, results_hard3.Support_Vector),
    "XGBOOST": accuracy_score(y_test, results_hard3.XGBOOST)
}
metrics3["Precision"] = {
    "Naive_Bayes": precision_score(y_test, results_hard3.Naive_Bayes),
    #"Support_Vector": precision_score(y_test, results_hard3.Support_Vector),
    "XGBOOST": precision_score(y_test, results_hard3.XGBOOST)
}
metrics3["Recall"] = {
    "Naive_Bayes": recall_score(y_test, results_hard3.Naive_Bayes),
    #"Support_Vector": recall_score(y_test, results_hard3.Support_Vector),
    "XGBOOST": recall_score(y_test, results_hard3.XGBOOST)
}
metrics3["F1"] = {
    "Naive_Bayes": f1_score(y_test, results_hard3.Naive_Bayes),
    #"Support_Vector": f1_score(y_test, results_hard3.Support_Vector),
    "XGBOOST": f1_score(y_test, results_hard3.XGBOOST)
}

metrics3 = pd.DataFrame(metrics3)
metrics3

## ROC Curves

In [ ]:
# Datos de FPR y TPR para los dos modelos 
fpr_Naive_Bayes3,tpr_Naive_Bayes3,_ = roc_curve(y_test, results_soft3.Naive_Bayes)
#fpr_Support_Vector3,tpr_Support_Vector3,_ = roc_curve(y_test, results_soft3.Support_Vector)
fpr_XGBOOST3,tpr_XGBOOST3,_ = roc_curve(y_test, results_soft3.XGBOOST)

# Calcular el área bajo la curva ROC (AUC) para cada modelo
auc_Naive_Bayes3 = auc(fpr_Naive_Bayes3,tpr_Naive_Bayes3)
#auc_Support_Vector3 = auc(fpr_Support_Vector3,tpr_Support_Vector3)
auc_XGBOOST3 = auc(fpr_XGBOOST3,tpr_XGBOOST3)

In [ ]:
# Crear la gráfica ROC
plt.figure(figsize=(8, 6))

# Graficar las curvas ROC para los tres modelos
plt.plot(fpr_Naive_Bayes3,tpr_Naive_Bayes3, label=f'Naive bayes (AUC = {auc_Naive_Bayes3:.2f})')
plt.plot(fpr_XGBOOST3,tpr_XGBOOST3, label=f'Xgboost (AUC = {auc_XGBOOST3:.2f})')
#plt.plot(fpr_Support_Vector3,tpr_Support_Vector3, label=f'Support Vector (AUC = {auc_Support_Vector3:.2f})')


# Configurar la gráfica
plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Línea diagonal para referencia
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC de Modelos')
plt.legend(loc="lower right")

# Mostrar la gráfica
plt.show()

In [ ]:
# ¡¡¡¡¡¡¡¡¡¡NOOOOOOOOOO LO CORRAS QUE SE EXPLOTA!!!!!!!!!!!
#target = list()
#frames = list()
#for i in data:
#    target=i['target']
#    for j in data:
#        frames.append(j['series'])
#    df = pd.concat(frames,axis = 1)
#    df['target'] = target